In [81]:
import json
import re
from os.path import join
import os

import pandas as pd
import numpy as np
from tqdm import tqdm

import cfbd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

with open("../config.json") as f:
    configuration = cfbd.Configuration(
        access_token = json.load(f)["CFBD_API_KEY"]
    )

In [97]:
YEARS = range(2023, 2024)
DATA_DIR = "../data"

In [98]:
def convert_to_snake_case(cols):
    cols_new = []
    for c in cols:
        s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', c)
        cols_new.append(re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower())
    return cols_new

In [ ]:
# Load games data
all_dfs = []
id_cols = ['id', 'season', 'week', 'season_type', 'completed', 'neutral_site', 'venue_id']
home_cols = ['home_id', 'home_team', 'home_conference', 'home_division', 'home_points', 'home_pregame_elo']
away_cols = ['away_id', 'away_team', 'away_conference', 'away_division', 'away_points', 'away_pregame_elo']

games_dir = join(DATA_DIR, "games")
if not os.path.exists(games_dir):
    os.makedirs(games_dir)


for year in YEARS:
    file_path = join(games_dir, f"{year}.parquet")
    if os.path.exists(file_path):
        print(f"Reading {file_path} from cached data")
        df_tmp = pd.read_parquet(file_path)
    else:
        print(f"Fetching {year} games data from CFBD API")
        with cfbd.ApiClient(configuration) as api_client:
            api_instance = cfbd.GamesApi(api_client)
            data = api_instance.get_games(year=year)
        df_tmp = pd.DataFrame([val.to_dict() for val in data])
        df_tmp.columns = convert_to_snake_case(df_tmp.columns)
        df_tmp = df_tmp[id_cols + [
                            'home_id','home_team','home_conference','home_classification','home_points','home_pregame_elo',
                            'away_id','away_team','away_conference','away_classification','away_points','away_pregame_elo']]
        df_tmp['season_type'] = df_tmp['season_type'].apply(lambda x: x.value)
        df_tmp = df_tmp.assign(
            home_division=df_tmp['home_classification'].apply(lambda x: x.value if x is not None else None),
            away_division=df_tmp['away_classification'].apply(lambda x: x.value if x is not None else None)
        ).drop(columns=['home_classification','away_classification'])
        df_tmp = df_tmp[id_cols + home_cols + away_cols]
        df_tmp.to_parquet(file_path)
    all_dfs.append(df_tmp)
del df_tmp

df_games = pd.concat(all_dfs, ignore_index=True).reset_index(drop=True)
df_games.head()

Reading ../data/games/2023.parquet from cached data


,id,season,week,season_type,completed,neutral_site,venue_id,home_id,home_team,home_conference,home_division,home_points,home_pregame_elo,away_id,away_team,away_conference,away_division,away_points,away_pregame_elo
0,401525434,2023,1,regular,True,True,3504.0,87,Notre Dame,FBS Independents,fbs,42.0,1733.0,2426,Navy,American Athletic,fbs,3.0,1471.0
1,401540199,2023,1,regular,True,True,3689.0,2382,Mercer,Southern,fcs,17.0,NaN,2453,North Alabama,UAC,fcs,7.0,NaN
2,401520145,2023,1,regular,True,False,3662.0,55,Jacksonville State,Conference USA,fbs,17.0,1500.0,2638,UTEP,Conference USA,fbs,14.0,1369.0
3,401525450,2023,1,regular,True,False,3601.0,166,New Mexico State,Conference USA,fbs,30.0,1322.0,113,Massachusetts,FBS Independents,fbs,41.0,1074.0
4,401540628,2023,1,regular,True,False,4419.0,399,UAlbany,CAA,fcs,34.0,NaN,2230,Fordham,Patriot,fcs,13.0,NaN


In [121]:
# Load plays data
all_dfs = []
cols = ['id', 'drive_id', 'game_id', 'drive_number', 'play_number', 'offense',
       'offense_conference', 'offense_score', 'defense', 'home', 'away',
       'defense_conference', 'defense_score', 'period', 'offense_timeouts',
       'defense_timeouts', 'yardline', 'yards_to_goal', 'down', 'distance',
       'yards_gained', 'scoring', 'play_type', 'play_text', 'ppa',
       'clock_minutes', 'clock_seconds']

plays_dir = join(DATA_DIR, "plays")
if not os.path.exists(plays_dir):
    os.makedirs(plays_dir)

for year in YEARS:
    season_file_path = join(plays_dir, f"{year}.parquet")
    
    if os.path.exists(season_file_path):
        print(f"Reading {season_file_path} from cached data")
        df_season_plays = pd.read_parquet(season_file_path)
    else:
        print(f"Fetching all weeks' plays data for {year} from CFBD API")
        weeks = df_games.query('season == @year').week.unique().tolist()

        season_dfs = []
        for week in weeks:
            print(f"Fetching {year} plays data for week {week} from CFBD API")
            with cfbd.ApiClient(configuration) as api_client:
                api_instance = cfbd.PlaysApi(api_client)
                data = api_instance.get_plays(year=year, week=week)
            df_tmp = pd.DataFrame([val.to_dict() for val in data])
            df_tmp.columns = convert_to_snake_case(df_tmp.columns)

            df_tmp["clock_minutes"] = df_tmp["clock"].apply(lambda x: x["minutes"])
            df_tmp["clock_seconds"] = df_tmp["clock"].apply(lambda x: x["seconds"])
            df_tmp.drop(columns=["clock"], inplace=True)

            df_tmp = df_tmp[cols]
            season_dfs.append(df_tmp)
        
        df_season_plays = pd.concat(season_dfs, ignore_index=True).reset_index(drop=True)
        
        df_season_plays.to_parquet(season_file_path)
        print(f"Saved plays data for {year} to {season_file_path}")
        del df_season_plays, season_dfs, df_tmp

    all_dfs.append(df_season_plays)
    
df_plays = pd.concat(all_dfs, ignore_index=True).reset_index(drop=True)
df_plays.head()

Reading ../data/plays/2023.parquet from cached data


,id,drive_id,game_id,drive_number,play_number,offense,offense_conference,offense_score,defense,home,away,defense_conference,defense_score,period,offense_timeouts,defense_timeouts,yardline,yards_to_goal,down,distance,yards_gained,scoring,play_type,play_text,ppa,clock_minutes,clock_seconds
0,401540244104995401,40154024433,401540244,22,6,Idaho,Big Sky,42,Lamar,Lamar,Idaho,Southland,17,4,3.0,3.0,67,67,3,28,5,False,Rush,Carlos Matheney run for 5 yds to the LAM 39,0.003424,0,45
1,401540244104994901,40154024433,401540244,22,5,Idaho,Big Sky,42,Lamar,Lamar,Idaho,Southland,17,4,3.0,3.0,66,66,2,27,-1,False,Rush,Carlos Matheney run for a loss of 1 yard to th...,-0.145843,0,50
2,401540244104977803,40154024433,401540244,22,4,Idaho,Big Sky,42,Lamar,Lamar,Idaho,Southland,17,4,3.0,3.0,64,64,1,25,-2,False,Rush,Carlos Matheney run for a loss of 2 yards to t...,-0.464241,2,21
3,401540244104977802,40154024433,401540244,22,3,Idaho,Big Sky,42,Lamar,Lamar,Idaho,Southland,17,4,3.0,3.0,49,49,1,10,-15,False,Rush,"Idaho Penalty, face mask (15 yards) (Carlos Ma...",-3.323128,2,21
4,401540244104977801,40154024433,401540244,22,2,Idaho,Big Sky,42,Lamar,Lamar,Idaho,Southland,17,4,3.0,3.0,65,65,3,10,16,False,Rush,Jack Layne run for 16 yds to the LAM 49 for a ...,3.095490,2,21


## Filters
1. Disclude overtime plays
2. Disculde plays with a penalty
3. Drop games without complete game play data (pct_game_played < 0.99)

In [26]:
df = df.query('period <= 4')

#TODO: disclude penalty plays

## Feature Engineering

In [31]:
df = df.assign(
    pct_game_played = (((df['period'] - 1) * 15 * 60) + df['clock_minutes'] * 60 + df['clock_seconds']) / (4 * 15 * 60),
    score_diff = df['offense_score'] - df['defense_score']
)

In [ ]:
final_scores = (
    df
    .sort_values(by=['period', 'pct_game_played'], ascending=[False, False])
    .drop_duplicates(subset=['game_id'], keep='first')
    .query('pct_game_played >= 0.99')
    [['game_id', 'offense', 'score_diff']]
)

if 'score_diff_final' in df.columns:
    df = df.drop(columns=['score_diff_final'])
if 'offense_final' in df.columns:
    df = df.drop(columns=['offense_final'])
df = (
    df
    .merge(final_scores, on='game_id', suffixes=('', '_final'))
    .dropna(subset=['score_diff_final'])
)

In [56]:
df.drop_duplicates('game_id')[['game_id','offense','defense','offense_final','score_diff_final']].head()

,game_id,offense,defense,offense_final,score_diff_final
0,401540244,Idaho,Lamar,Idaho,32
33,401525435,Notre Dame,Tennessee State,Tennessee State,-46
111,401551798,Notre Dame,Oregon State,Notre Dame,24
147,401525434,Notre Dame,Navy,Notre Dame,35
266,401551749,Oklahoma State,Texas A&M,Texas A&M,-11


In [63]:
df_games.query('id==401525434')[['home_team','home_points','away_team','away_points']]

,home_team,home_points,away_team,away_points
0,Notre Dame,42,Navy,3
